In [6]:
import os
import pickle
import numpy as np
from methods import simple_bump, square_bump, solver, D2_per, D4_per, D6_per

c0 = 1
power = 4

h = 0.005
X = np.arange(0, 1+h, h)

x0 = 0.25
x1 = 0.75
simple = np.array([simple_bump(x, x0, x1, power, c0) for x in X])

x0 = 0.25
x1 = 0.35
x2 = 0.65
x3 = 0.75
square = np.array([square_bump(x, x0, x1, x2, x3, power, c0) for x in X])


def solver(f, k, h, ti, tf, x0, samples='all', method='RK4', verbose=False):
       
    Nt = round((tf - ti) / k) + 1
    print(Nt)
    grid = np.arange(0, 1+h, h)
        
    x = x0
    data = [x]
    time = [ti]
    t = ti + k
    #for i, t in enumerate(time[1:], start=1):
    i = 1
    while t <= tf:

        if method == 'euler':
            x = x + k * f(x, t)
        elif method == 'RK4':
            q1 = f(x, t)
            q2 = f(x + (k/2)*q1, t+(k/2))
            q3 = f(x + (k/2)*q2, t+(k/2))
            q4 = f(x + k*q3, t+k)
            x = x + (k/6)*(q1 + 2*q2 + 2*q3 + q4) 
        
        #if samples == 'all' or ((samples*i) % (Nt-1)) == 0:
        if samples == 'all' or ((samples*i) % (Nt-1)) == 0:
            if verbose:
                print('{:2f}'.format(t))
            time.append(t)
            data.append(x)
        i += 1
        t += k
    
    time = np.array(time)
    data = np.array(data)
    assert(time.size*grid.size == data.size)
    return time, grid, data

v = -1
#k = 0.0001
k = 0.00001
ti = 0
T = 1
C = np.abs(v*k/h)

def f(x, t):
    return v*D6_per(x, h)

Dx_dict = {
    'D2': D2_per,
    'D4': D4_per,
    'D6': D6_per
}
overwrite = True
bumps = ['simple', 'square']
samples = 100
sols = {}
for bump in bumps:
    print(bump)
    sols[bump] = {}
    if bump == 'simple':
        x0 = simple
    else:
        x0 = square
    for key, Dx in Dx_dict.items():

        pickle_file = '{}_k{}_h{}_T{}_samples{}_{}.pickle'.format(bump, k, h, T, samples, key)

        print(key)
        func = lambda x, t: v*Dx(x, h)
        if os.path.isfile(pickle_file) and not overwrite:
            data = pickle.load(open(pickle_file, 'rb'))
            time, grid, sol = data
        else:
            time, grid, sol = solver(func, k, h, ti, T, x0, samples=samples, method='euler', verbose=False)
            pickle.dump([time, grid, sol], open(pickle_file, 'wb'))
        sols[bump][key] = sol
    
Nx = grid.size
Nt = time.size

print('CFL =', C)
print('Nx =', Nx)
print('Nt =', Nt)
print('sol shape =', sol.shape)
print('time[-1] =', time[-1])
print('time[-2] =', time[-2])

simple
D2
100001
D4
100001
D6
100001
square
D2
100001
D4
100001
D6
100001
CFL = 0.002
Nx = 201
Nt = 101
sol shape = (101, 201)
time[-1] = 0.9999999999980838
time[-2] = 0.9899999999981293
